In [1]:
import gurobipy as gp
from gurobipy import GRB
import random

# Generate 10 random crystals with coordinates between 0 and 10 (original comment said 0 to 10000) 
# and positive values between 1 and 1000.
crystals = [(random.randint(0, 10), random.randint(0, 10), random.randint(1, 1000)) for _ in range(100)]

# Generate 10 random mines with coordinates between 0 and 10 (original comment said 0 to 10000) 
# and negative values between -1000 and -1.
mines = [(random.randint(0, 10), random.randint(0, 10), random.randint(-1000, -1)) for _ in range(100)]


# ---------------------------
# Parameters and Model Setup
# ---------------------------
n = 6                     # Total number of vertices for the polygon
N = 6                     # Number of vertices (and edges) of the polygon
M = 100                    # Big-M constant, equal to the coordinate upper bound
eps = 1                  # A small positive gap to ensure strict separation between edges/points

model = gp.Model("polygon_nonoverlap")  # Initialize the Gurobi model

# ------------------------------------------
# 1. Define Vertex Coordinates and Edge Orientation
# ------------------------------------------
# Create integer vertex coordinates in [0, M]
x = model.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=M, name="x")  # x-coordinates for vertices
y = model.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=M, name="y")  # y-coordinates for vertices

# For each edge i (from vertex i to vertex (i+1) mod n),
# define binary variable z[i]:
#   z[i] = 1 means edge i is horizontal (enforcing y[i] == y[i+1])
#   z[i] = 0 means edge i is vertical   (enforcing x[i] == x[i+1])
z = model.addVars(n, vtype=GRB.BINARY, name="z")
z1 = model.addVars(n, vtype=GRB.BINARY, name="z")  # Additional binary variables used in constraints
z2 = model.addVars(n, vtype=GRB.BINARY, name="z")  # Additional binary variables used in constraints

for i in range(n):
    j = (i + 1) % n  # j is the next vertex (with wrap-around)
    # Enforce horizontal: if z[i]==1 then y[i] == y[j]
    model.addConstr(z[i] == 1 - z[j])  # Forces alternating orientation between consecutive edges
    model.addConstr(y[i] - y[j] <= M * (1 - z[i]), name=f"hor_y1_{i}")  # If z[i]==1, forces y[i]==y[j]
    model.addConstr(y[j] - y[i] <= M * (1 - z[i]), name=f"hor_y2_{i}")  # Complementary constraint for horizontal edge
    # Additional constraints for horizontal edges when z[i]==1: enforce a gap using eps and auxiliary binary z1
    model.addConstr(x[i] - x[j] + eps <= M * (1 - z[i]) + M * (z1[i]), name=f"vert_x1_{i}")
    model.addConstr(x[j] - x[i] + eps <= M * (1 - z[i]) + M * (1 - z1[i]), name=f"vert_x2_{i}")
    # Enforce vertical: if z[i]==0 then x[i] == x[j]
    model.addConstr(x[i] - x[j] <= M * z[i], name=f"vert_x1_{i}")  # If z[i]==0, forces x[i]==x[j]
    model.addConstr(x[j] - x[i] <= M * z[i], name=f"vert_x2_{i}")  # Complementary constraint for vertical edge
    # Additional constraints for vertical edges when z[i]==0: enforce a gap using eps and auxiliary binary z2
    model.addConstr(y[i] - y[j] + eps <= M * (z[i]) + M * (z2[i]), name=f"hor_y1_{i}")
    model.addConstr(y[j] - y[i] + eps <= M * (z[i]) + M * (1 - z2[i]), name=f"hor_y2_{i}")

# ------------------------------------------
# 2. Define Auxiliary Variables for Edge Endpoints (Corrected)
# ------------------------------------------
# For each edge i (with j = (i+1)%n) we compute:
#   - For horizontal edges: L[i] = min(x[i], x[j]) and R[i] = max(x[i], x[j])
#   - For vertical edges:   B[i] = min(y[i], y[j]) and T[i] = max(y[i], y[j])
#
# Here we use Gurobi’s general constraints to define the min and max.
L = {}  # Dictionary for left x-coordinate of edge i
R = {}  # Dictionary for right x-coordinate of edge i
B = {}  # Dictionary for bottom y-coordinate of edge i
T = {}  # Dictionary for top y-coordinate of edge i

for i in range(n):
    j = (i + 1) % n
    # For horizontal edges, define L[i] and R[i]
    L[i] = model.addVar(lb=0, ub=M, vtype=GRB.INTEGER, name=f"L_{i}")  # Left endpoint x-value
    R[i] = model.addVar(lb=0, ub=M, vtype=GRB.INTEGER, name=f"R_{i}")  # Right endpoint x-value
    model.addGenConstrMin(L[i], [x[i], x[j]], name=f"min_x_{i}")       # L[i] is the minimum of x[i] and x[j]
    model.addGenConstrMax(R[i], [x[i], x[j]], name=f"max_x_{i}")       # R[i] is the maximum of x[i] and x[j]
    
    # For vertical edges, define B[i] and T[i]
    B[i] = model.addVar(lb=0, ub=M, vtype=GRB.INTEGER, name=f"B_{i}")  # Bottom endpoint y-value
    T[i] = model.addVar(lb=0, ub=M, vtype=GRB.INTEGER, name=f"T_{i}")  # Top endpoint y-value
    model.addGenConstrMin(B[i], [y[i], y[j]], name=f"min_y_{i}")       # B[i] is the minimum of y[i] and y[j]
    model.addGenConstrMax(T[i], [y[i], y[j]], name=f"max_y_{i}")       # T[i] is the maximum of y[i] and y[j]


for i in range(n):
    for j in range(i+1, n):
        # Skip adjacent edges (touching is allowed)
        if j == i+1 or (i == 0 and j == n-1):
            continue

        # Create binary variables delta and delta1 for non-overlap constraints between non-adjacent edges
        delta = model.addVar(vtype=GRB.BINARY, name=f"delta_{i}_{j}")
        delta1 = model.addVar(vtype=GRB.BINARY, name=f"delta1_{i}_{j}")
        # Non-overlap constraints for horizontal edges: ensure proper separation using eps and Big-M method
        model.addConstr(R[i] + eps <= x[j] + M*(1 - z[i] + z[j] + delta + delta1),
                        name=f"hor_nonoverlap1_{i}_{j}")
        model.addConstr(L[i] - eps >= x[j] - M*(1 - z[i] + z[j] + (1 - delta) + delta1),
                        name=f"hor_nonoverlap2_{i}_{j}")
        model.addConstr(y[i] + eps <= B[j] + M*(1 - z[i] + z[j] + 1 - delta1 + delta),
                        name=f"hor_nonoverlap1_{i}_{j}")
        model.addConstr(y[i] - eps >= T[j] - M*(1 - z[i] + z[j] + 1 - delta1 + 1 - delta),
                        name=f"hor_nonoverlap1_{i}_{j}")
 
        # Create additional binary variables eta and eta1 for alternative non-overlap conditions (symmetry)
        eta = model.addVar(vtype=GRB.BINARY, name=f"eta_{i}_{j}")
        eta1 = model.addVar(vtype=GRB.BINARY, name=f"eta1_{i}_{j}")
        model.addConstr(R[j] + eps <= x[i] + M*(1 + z[i] - z[j] + eta + eta1),
                        name=f"hor_nonoverlap1_{i}_{j}")
        model.addConstr(L[j] - eps >= x[i] - M*(1 + z[i] - z[j] + (1 - eta) + eta1),
                        name=f"hor_nonoverlap2_{i}_{j}")
        model.addConstr(y[j] + eps <= B[i] + M*(1 + z[i] - z[j] + 1 - eta1 + eta),
                        name=f"hor_nonoverlap1_{i}_{j}")
        model.addConstr(y[j] - eps >= T[i] - M*(1 + z[i] - z[j] + 1 - eta1 + 1 - eta),
                        name=f"hor_nonoverlap1_{i}_{j}")


# ------------------------------------------
# 3. Inside-Outside Point Classification using ray projection method
# ------------------------------------------
I = model.addVars(len(crystals) + len(mines), vtype=GRB.BINARY, name="I")  # Binary variable: 1 if point is inside, 0 otherwise
k = model.addVars(len(crystals) + len(mines), vtype=GRB.INTEGER, name="k") # Integer variable used in crossing number calculation
odd = model.addVars(len(crystals) + len(mines), vtype=GRB.BINARY, name="Odd")  # Binary variable indicating odd parity of crossings        
for idx, (px, py, _) in enumerate(crystals + mines):
    crossing_count = gp.LinExpr()  # Expression to count edge crossings for point idx
    edge_count = gp.LinExpr()      # Expression to count edges meeting certain criteria for point idx
    for i in range(N):
        next_i = (i + 1) % N  # Next vertex index for the edge
        # Create binary variables for determining if the current edge contributes to crossings
        e = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        e1 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        e2 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        e3 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        e4 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        f = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        f1 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        f2 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        f3 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        f4 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
        f5 = model.addVar(vtype=GRB.BINARY, name=f"cross_{idx}_{i}")
     
        model.addConstr(1 - z[i] == e1)  # Sets e1 based on the edge orientation (if not horizontal)
        model.addConstr(z[i] == f1)        # Sets f1 based on the edge orientation (if horizontal)
        # Constraint to check if y_p (point's y) is between y[i] and y[next_i] using B and T values
        model.addConstr(B[i] - py + eps <= M * (1 - e2))
        model.addConstr(B[i] - py >= -M * (e2))
        model.addConstr(T[i] - py >= -M * (1 - e3))
        model.addConstr(T[i] - py + eps <= M * (1 - e3))
        # Constraints to relate vertex y-coordinate with point's y-coordinate using auxiliary binary variables f2 and f3
        model.addConstr(y[i] >= py - M * (1 - f2))
        model.addConstr(y[i] + eps <= py + M * (f2))
        model.addConstr(y[i] <= py + M * (1 - f3))
        model.addConstr(y[i] - eps >= py - M * (f3))
        # Constraint to check if the edge is to the right of the point's x-coordinate using e4
        model.addConstr(x[i] - px - eps >= 0 - M * (1 - e4))
        model.addConstr(x[i] - px <= 0 + M * (e4))
        # Constraints using L and R to further define the edge's relation to the point's x-coordinate using f4, f1, and f5
        model.addConstr(L[i] <= px + M * (1 - f4 + 1 - f1))
        model.addConstr(L[i] - eps >= px - M * (f4 + 1 - f1))
        model.addConstr(R[i] >= px - M * (1 - f5 + 1 - f1))
        model.addConstr(R[i] + eps <= px + M * (f5 + 1 - f1))
        
        # Sum conditions to determine if edge i is counted as a crossing for point idx
        model.addConstr(e1 + e2 + e3 + e4 >= 4 - M * (1 - e))
        model.addConstr(e1 + e2 + e3 + e4 <= 3 + M * (e))
        model.addConstr(f1 + f2 + f3 + f4 + f5 >= 5 - M * (1 - f))
        model.addConstr(f1 + f2 + f3 + f4 + f5 <= 4 + M * (f))

        # If all conditions are satisfied, this edge is counted as a crossing
        crossing_count += e
        edge_count += f
    # Modulo replacement: crossing_count = 2 * k[idx] + odd[idx] (parity check for odd number of crossings)
    model.addConstr(crossing_count == 2 * k[idx] + odd[idx])
    model.addConstr(edge_count + odd[idx] >= 1 - M * (1 - I[idx]))
    model.addConstr(edge_count + odd[idx] <= 0 + M * (I[idx]))
    # If crossings are odd, the point is considered inside the polygon


# Objective: Maximize total enclosed value (reward for crystals, penalty for mines)
model.update()
model.setObjective(
    gp.quicksum(I[i] * crystals[i][2] for i in range(len(crystals))) -
    gp.quicksum(I[i + len(crystals)] * abs(mines[i][2]) for i in range(len(mines))),
    GRB.MAXIMIZE
)
# ------------------------------------------
# 4. Optimize and Print the Selected Vertices
# ------------------------------------------
# Optimize the model (feasible solution is sought based on the constraints and objective)
model.optimize()

if model.status == GRB.OPTIMAL:
    for i in range(N):
        print(f"Vertex {i}: ({x[i].X}, {y[i].X})")  # Print coordinates of each vertex
    for idx, (px, py, val) in enumerate(crystals + mines):
        status = "Inside" if I[idx].X > 0.5 else "Outside"  # Determine if each point is inside the polygon
        print(f"Point ({px}, {py}, value={val}) is {status}")
else:
    print("No feasible solution found.")

Set parameter Username
Set parameter LicenseID to value 2617099
Academic license - for non-commercial use only - expires 2026-02-01
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24726 rows, 13890 columns and 75012 nonzeros
Model fingerprint: 0xa26d2bb6
Model has 24 simple general constraints
  12 MAX, 12 MIN
Variable types: 12 continuous, 13878 integer (13654 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 1e+03]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 3e+02]
Presolve removed 10620 rows and 6790 columns
Presolve time: 0.17s
Presolved: 14106 rows, 7100 columns, 43536 nonzeros
Variable types: 60 continuous, 7040 integer (7016 binary)

Root relaxation: objective 4.953700e+04, 2642 iterations, 0.04 seconds (0.11 work units)

    Nodes    |    Current Node    |     Objec